In [1]:
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
from urllib import parse
import time, requests, datetime, re, json, os
import pandas as pd
from metadata import *
import numpy as np

def doScrollDown(whileSeconds):
        start = datetime.datetime.now()
        end = start + datetime.timedelta(seconds=whileSeconds)
        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            time.sleep(1.5)
            if datetime.datetime.now() > end:
                break

keyword = input("검색어: ")


driver = wd.Chrome(driver_path)
print(f"login start - option {login_option}")
login_url = "https://www.instagram.com/accounts/login/"
driver.get(login_url)
time.sleep(5)

is_login_success = False
if login_option == "instagram":
    try:
        instagram_id_form = driver.find_element(By.XPATH, instagram_id_name)
        instagram_id_form.send_keys(user_id)
        time.sleep(3)

        instagram_pw_form = driver.find_element(By.XPATH, instagram_pw_name)
        instagram_pw_form.send_keys(user_pw)
        time.sleep(3)
        
        login_ok_button =  driver.find_element(By.XPATH, instagram_login_btn)
        login_ok_button.click()
        is_login_success = True
    except:
        print("instagram login fail")
        is_login_success = False
    time.sleep(10)


if is_login_success:
    print(f"login {login_option} success")
    print(f"Start {keyword} Extract")
    # keyword = parse.quote(keyword)
    url = f"https://www.instagram.com/explore/tags/{keyword}/"
    driver.get(url)
    time.sleep(5)
    
    # 스크롤 다운
    doScrollDown(10)

# 본문 url
html = driver.page_source
soup = bs(html, 'html.parser')
soup2 = soup.find("article", class_="_aao7")
soup3 = soup2.find_all("div", class_="_aabd _aa8k _aanf")
print(len(soup3),"건")
for i in soup3:
    try:
        driver.get("https://www.instagram.com/"+i.a['href'])
        time.sleep(2)
        html = driver.page_source
        main_soup = bs(html, 'html.parser')
        
        # 태그
        main_soup_tag = main_soup.find_all("div", class_="_a9zs" )
        try:
            try:
                tags = re.findall('#[A-Za-z0-9가-힣]+', str(main_soup_tag[0]))
            except:
                tags = re.findall('#[A-Za-z0-9가-힣]+', str(main_soup_tag[1]))
        except:
            tags = np.nan
        tag_data = "".join(tags)
        tag.append(tag_data.replace("#"," "))
        
        # 업로더
        main_soup_id = main_soup.find_all("div", class_="_aaqt")
        upload_id.append(main_soup_id[0].text)
        
        # 이미지
        try:
            img_src = main_soup.find_all("img", class_="_aagt")
            img_url.append(img_src[0].attrs['src'])
        except:
            img_url.append(np.nan)
    except:
        pass
driver.close()
insta_crawling_df = pd.DataFrame({"upload_id": upload_id,
                                                        "tag": tag,
                                                        "img_url": img_url})
# 저장
num = 1
while True:
    if not os.path.exists(f"./data/insta_crawling_{keyword}({num}).csv"):
        insta_crawling_df.to_csv(f"./data/insta_crawling_{keyword}({num}).csv", index=False)
        break
    num += 1

login start - option instagram
login instagram success
Start polaris.nail Extract


AttributeError: 'NoneType' object has no attribute 'find_all'

In [21]:
len(upload_id),len(tag),len(img_url)

(361, 361, 361)

In [18]:
insta_crawling_df.head()

,upload_id,tag,img_url
0,zeze._nail,두정동 겨울네일 nail 네일교육 키치네일 뷰티인재양성교육센터 아르코왁싱충북센터 ...,https://scontent-ssn1-1.cdninstagram.com/v/t51...
1,zeze._nail,두정동 겨울네일 nail 네일교육 키치네일 뷰티인재양성교육센터 아르코왁싱충북센터 ...,https://scontent-ssn1-1.cdninstagram.com/v/t51...
2,zeze._nail,두정동 겨울네일 nail 네일교육 키치네일 뷰티인재양성교육센터 아르코왁싱충북센터 ...,https://scontent-ssn1-1.cdninstagram.com/v/t51...
3,zeze._nail,두정동 겨울네일 nail 네일교육 키치네일 뷰티인재양성교육센터 아르코왁싱충북센터 ...,https://scontent-ssn1-1.cdninstagram.com/v/t51...
4,zeze._nail,두정동 겨울네일 nail 네일교육 키치네일 뷰티인재양성교육센터 아르코왁싱충북센터 ...,https://scontent-ssn1-1.cdninstagram.com/v/t51...
